# 02 - Data Models

## Agentic Logistics Control System

This notebook documents and demonstrates the data models used throughout the system:
- Pydantic models for data validation
- LangGraph state schemas
- Enums for type safety

### Control Loop Role: Data Structures

These models are used across all phases of the control loop:
```
OBSERVE -> REASON -> PLAN -> DECIDE -> ACT -> FEEDBACK -> (loop)
```

In [ ]:
# Setup path
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

In [ ]:
# Import all models
from src.models import (
    # Enums
    TruckStatus, TrafficLevel, LoadPriority, ActionType, ControlLoopPhase,
    # Core models
    Location, GPSReading, Truck, Route, RoutePoint, Load, TrafficCondition,
    # Analysis models
    Issue, ReasoningResult,
    # Planning models
    Scenario, PlanningResult,
    # Decision models
    Decision, DecisionResult,
    # Action models
    ActionResult, Notification,
    # Feedback models
    OutcomeMetrics, LearningUpdate, FeedbackResult,
    # State
    AgentState, SystemState,
)
from datetime import datetime, timedelta

print("All models imported successfully!")

## 1. Core Entity Models

In [ ]:
# Create sample locations
warehouse = Location(
    latitude=40.7128,
    longitude=-74.0060,
    name="Main Warehouse",
    address="123 Logistics Way, New York, NY"
)

customer_site = Location(
    latitude=40.7580,
    longitude=-73.9855,
    name="Customer A",
    address="456 Delivery St, New York, NY"
)

# Calculate distance
distance = warehouse.distance_to(customer_site)
print(f"Distance: {distance:.2f} km")
print(f"\nWarehouse: {warehouse.model_dump_json(indent=2)}")

In [ ]:
# Create a truck
truck = Truck(
    id="TRK-001",
    name="Alpha Hauler",
    status=TruckStatus.EN_ROUTE,
    current_location=Location(latitude=40.7300, longitude=-74.0000),
    driver_id="DRV-042",
    capacity_kg=15000,
    fuel_level_percent=75.5,
    last_gps_reading=GPSReading(
        truck_id="TRK-001",
        location=Location(latitude=40.7300, longitude=-74.0000),
        speed_kmh=45.0,
        heading=45.0
    )
)

print(f"Truck Status: {truck.status.value}")
print(f"\nTruck Details:\n{truck.model_dump_json(indent=2)}")

In [ ]:
# Create a load
load = Load(
    id="LOAD-2024-001",
    description="Electronics shipment - fragile",
    weight_kg=2500,
    volume_m3=12.5,
    priority=LoadPriority.HIGH,
    pickup_location=warehouse,
    delivery_location=customer_site,
    delivery_deadline=datetime.utcnow() + timedelta(hours=4),
    assigned_truck_id="TRK-001"
)

print(f"Load Priority: {load.priority.value}")
print(f"Deadline: {load.delivery_deadline}")

In [ ]:
# Create a route
route = Route(
    id="ROUTE-001",
    truck_id="TRK-001",
    origin=warehouse,
    destination=customer_site,
    waypoints=[
        RoutePoint(
            location=Location(latitude=40.7200, longitude=-74.0000),
            sequence=1,
            is_waypoint=True
        ),
        RoutePoint(
            location=customer_site,
            sequence=2,
            is_destination=True
        )
    ],
    estimated_distance_km=distance,
    estimated_duration_minutes=45,
    estimated_fuel_consumption_liters=distance * 0.3,
    started_at=datetime.utcnow()
)

print(f"Route: {route.id}")
print(f"Estimated Duration: {route.estimated_duration_minutes} minutes")

In [ ]:
# Create traffic condition
traffic = TrafficCondition(
    segment_id="SEG-I95-NB-42",
    level=TrafficLevel.HEAVY,
    speed_kmh=25,
    delay_minutes=15,
    incident_description="Multi-vehicle accident, right lane blocked",
    affected_routes=["ROUTE-001"]
)

print(f"Traffic Level: {traffic.level.value}")
print(f"Delay Factor: {traffic.delay_factor}x")
print(f"Delay: {traffic.delay_minutes} minutes")

## 2. Analysis and Reasoning Models

In [ ]:
# Create an issue detected by reasoning
issue = Issue(
    id="ISSUE-001",
    type="delay",
    severity="high",
    description="Truck TRK-001 delayed by 15+ minutes due to heavy traffic on I-95",
    affected_truck_ids=["TRK-001"],
    affected_load_ids=["LOAD-2024-001"],
    metadata={
        "traffic_segment": "SEG-I95-NB-42",
        "expected_delay_minutes": 15,
        "delivery_at_risk": True
    }
)

print(f"Issue: {issue.type} - {issue.severity}")
print(f"Description: {issue.description}")

In [ ]:
# Create reasoning result
reasoning = ReasoningResult(
    situation_summary="Fleet operating with one significant delay due to traffic incident",
    issues=[issue],
    risk_assessment="Medium-High: One delivery deadline at risk, customer notification recommended",
    recommendations=[
        "Consider alternative route via Route 9A",
        "Notify customer of potential delay",
        "Evaluate reassigning nearby truck TRK-003"
    ],
    confidence=0.85,
    reasoning_trace=[
        "Analyzed GPS data for 12 active trucks",
        "Detected speed anomaly for TRK-001 (25 km/h in 65 km/h zone)",
        "Cross-referenced with traffic API - confirmed incident",
        "Calculated impact on delivery deadline",
        "Generated recommendations based on fleet state"
    ]
)

print(f"Confidence: {reasoning.confidence}")
print(f"\nRecommendations:")
for rec in reasoning.recommendations:
    print(f"  - {rec}")

## 3. Planning and Scenario Models

In [ ]:
# Create scenarios
scenario_wait = Scenario(
    id="SCEN-001-WAIT",
    name="Wait for Traffic Clear",
    description="Continue on current route, wait for traffic to clear",
    actions=[{"type": "wait", "duration_minutes": 20}],
    estimated_cost=50,
    estimated_time_minutes=65,
    estimated_fuel_liters=2.5,
    reliability_score=0.7
)

scenario_reroute = Scenario(
    id="SCEN-001-REROUTE",
    name="Reroute via 9A",
    description="Take alternative route via Route 9A to avoid traffic",
    actions=[{"type": "reroute", "new_route": "9A-NORTH"}],
    estimated_cost=75,
    estimated_time_minutes=55,
    estimated_fuel_liters=4.2,
    reliability_score=0.9
)

scenario_reassign = Scenario(
    id="SCEN-001-REASSIGN",
    name="Reassign to TRK-003",
    description="Reassign load to nearby truck TRK-003",
    actions=[
        {"type": "reassign", "from_truck": "TRK-001", "to_truck": "TRK-003"},
        {"type": "dispatch", "truck": "TRK-003"}
    ],
    estimated_cost=150,
    estimated_time_minutes=40,
    estimated_fuel_liters=3.0,
    reliability_score=0.95
)

print("Scenarios generated:")
for s in [scenario_wait, scenario_reroute, scenario_reassign]:
    print(f"  {s.name}: {s.estimated_time_minutes} min, ${s.estimated_cost}, reliability {s.reliability_score}")

In [ ]:
# Create planning result
planning = PlanningResult(
    issue_id="ISSUE-001",
    scenarios=[scenario_wait, scenario_reroute, scenario_reassign],
    comparison_matrix={
        "SCEN-001-WAIT": {"time": 0.6, "cost": 0.9, "reliability": 0.7, "overall": 0.73},
        "SCEN-001-REROUTE": {"time": 0.8, "cost": 0.7, "reliability": 0.9, "overall": 0.80},
        "SCEN-001-REASSIGN": {"time": 0.95, "cost": 0.4, "reliability": 0.95, "overall": 0.77}
    },
    recommended_scenario_id="SCEN-001-REROUTE"
)

print(f"Recommended: {planning.recommended_scenario_id}")
print(f"\nComparison Matrix:")
for sid, scores in planning.comparison_matrix.items():
    print(f"  {sid}: overall={scores['overall']}")

## 4. Decision Models

In [ ]:
# Create a decision
decision = Decision(
    id="DEC-001",
    scenario_id="SCEN-001-REROUTE",
    action_type=ActionType.REROUTE,
    parameters={
        "truck_id": "TRK-001",
        "new_route": "9A-NORTH",
        "waypoints": [
            {"lat": 40.7350, "lng": -73.9900},
            {"lat": 40.7500, "lng": -73.9850}
        ]
    },
    score=0.80,
    confidence=0.85,
    rationale="Reroute via 9A offers best balance of time savings and reliability. "
              "Expected to save 10 minutes vs waiting, with 90% reliability.",
    llm_verified=True
)

print(f"Decision: {decision.action_type.value}")
print(f"Score: {decision.score}, Confidence: {decision.confidence}")
print(f"Rationale: {decision.rationale}")

In [ ]:
# Create decision result
decision_result = DecisionResult(
    selected_decision=decision,
    alternatives=[
        Decision(
            id="DEC-001-ALT1",
            scenario_id="SCEN-001-WAIT",
            action_type=ActionType.WAIT,
            parameters={"duration_minutes": 20},
            score=0.73,
            confidence=0.70,
            rationale="Lower cost but higher time risk"
        )
    ],
    requires_human_approval=False,
    decision_trace=[
        "Evaluated 3 scenarios against criteria",
        "REROUTE scored highest overall (0.80)",
        "Verified with Grok LLM - approved",
        "Confidence above threshold (0.85 > 0.70)"
    ]
)

print(f"Selected: {decision_result.selected_decision.action_type.value}")
print(f"Requires Human Approval: {decision_result.requires_human_approval}")

## 5. Action and Feedback Models

In [ ]:
# Create action result
action_result = ActionResult(
    action_id="ACT-001",
    decision_id="DEC-001",
    success=True,
    message="Route updated for TRK-001, driver notified",
    details={
        "navigation_updated": True,
        "driver_acknowledged": True,
        "ems_updated": True,
        "customer_notified": False  # Pending
    },
    rollback_possible=True
)

print(f"Action: {action_result.action_id}")
print(f"Success: {action_result.success}")
print(f"Message: {action_result.message}")

In [ ]:
# Create notification
notification = Notification(
    id="NOTIF-001",
    recipient_type="driver",
    recipient_id="DRV-042",
    subject="Route Update Required",
    message="Traffic incident ahead on I-95. Please follow updated route via 9A. "
            "New ETA: 45 minutes. Tap to acknowledge.",
    priority="high"
)

print(f"Notification to {notification.recipient_type}: {notification.subject}")

In [ ]:
# Create outcome metrics for feedback
outcome = OutcomeMetrics(
    decision_id="DEC-001",
    predicted_time_minutes=55,
    actual_time_minutes=52,
    predicted_cost=75,
    actual_cost=72,
    success=True,
    deviation_percent=-5.5  # Better than predicted
)

print(f"Predicted: {outcome.predicted_time_minutes} min")
print(f"Actual: {outcome.actual_time_minutes} min")
print(f"Deviation: {outcome.deviation_percent}%")

In [ ]:
# Create feedback result
feedback = FeedbackResult(
    outcomes=[outcome],
    learning_updates=[
        LearningUpdate(
            parameter_name="route_9a_speed_factor",
            old_value=1.0,
            new_value=1.05,
            reason="Route 9A consistently faster than model predicts"
        )
    ],
    system_health="healthy",
    recommendations=[
        "Update Route 9A speed model based on recent data",
        "Consider Route 9A as preferred alternative more often"
    ]
)

print(f"System Health: {feedback.system_health}")
print(f"Learning Updates: {len(feedback.learning_updates)}")

## 6. LangGraph State Schema

In [ ]:
from datetime import datetime
import uuid

# Create a sample AgentState
state: AgentState = {
    "current_phase": ControlLoopPhase.OBSERVE,
    "cycle_id": str(uuid.uuid4()),
    "trucks": [truck.model_dump()],
    "routes": [route.model_dump()],
    "loads": [load.model_dump()],
    "traffic_conditions": [traffic.model_dump()],
    "gps_readings": [],
    "observation_timestamp": datetime.utcnow().isoformat(),
    "reasoning_result": None,
    "current_issues": [],
    "planning_result": None,
    "scenarios": [],
    "decision_result": None,
    "selected_decision": None,
    "action_results": [],
    "notifications_sent": [],
    "feedback_result": None,
    "continue_loop": True,
    "requires_human_intervention": False,
    "error_message": None,
    "cycle_start_time": datetime.utcnow().isoformat(),
    "cycle_end_time": None,
    "total_cycles": 0,
}

print(f"AgentState created for cycle: {state['cycle_id']}")
print(f"Current phase: {state['current_phase']}")
print(f"Trucks: {len(state['trucks'])}")
print(f"Continue loop: {state['continue_loop']}")

## 7. System State (High-level)

In [ ]:
# Create SystemState
system_state = SystemState(
    trucks=[truck],
    routes=[route],
    loads=[load],
    traffic_conditions=[traffic],
    active_issues=[issue],
    pending_decisions=[decision],
    recent_actions=[action_result],
    recent_outcomes=[outcome],
    total_cycles_completed=42
)

print(f"System State:")
print(f"  Trucks: {len(system_state.trucks)}")
print(f"  Active Issues: {len(system_state.active_issues)}")
print(f"  Total Cycles: {system_state.total_cycles_completed}")

# Convert to AgentState
agent_state = system_state.to_agent_state()
print(f"\nConverted to AgentState with {len(agent_state['trucks'])} trucks")

## Model Validation Examples

In [ ]:
from pydantic import ValidationError

# Test validation
print("Testing model validation...")

# Invalid latitude
try:
    invalid_loc = Location(latitude=100, longitude=0)  # latitude > 90
except ValidationError as e:
    print(f"✓ Invalid latitude rejected: {e.errors()[0]['msg']}")

# Invalid truck capacity
try:
    invalid_truck = Truck(id="T1", name="Test", capacity_kg=-100)
except ValidationError as e:
    print(f"✓ Invalid capacity rejected: {e.errors()[0]['msg']}")

# Invalid confidence
try:
    invalid_decision = Decision(
        id="D1", scenario_id="S1", action_type=ActionType.WAIT,
        score=1.5, confidence=0.5, rationale="test"  # score > 1
    )
except ValidationError as e:
    print(f"✓ Invalid score rejected: {e.errors()[0]['msg']}")

print("\nAll validation tests passed!")

## Next Steps

1. Proceed to `03_perception_layer.ipynb` to implement data collection
2. These models will be used across all subsequent notebooks